In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
% matplotlib inline
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.layers import Conv1D,MaxPooling1D,LSTM
import pandas as pd

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
target = pd.read_csv('../input/sample_submission.csv')


X = train.iloc[:,2:].values
y = train.iloc[:,1].values
test = test.iloc[:,1:].values

scaler = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 3017)

scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)



dim_x_train = X_train_scaled.reshape(X_train_scaled.shape[0],X_train_scaled.shape[1],1)
dim_x_test = X_test_scaled.reshape(X_test_scaled.shape[0],X_test_scaled.shape[1],1)


In [ ]:
cmodel = Sequential()
cmodel.add(Conv1D(100,(1),input_shape=(200,1),activation="relu"))
# cmodel.add(Conv1D(100,(3),activation="relu"))
cmodel.add(BatchNormalization())
cmodel.add(Dense(700,activation="relu"))
cmodel.add(BatchNormalization())
cmodel.add(Dense(100,activation="relu"))
# cmodel.add(Dense(2,activation="relu"))

cmodel.add(Flatten())
cmodel.add(Dense(1,activation="sigmoid"))
optimizer = Nadam(1e-6)
es = EarlyStopping(patience=15, verbose=1)
cmodel.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = cmodel.fit(dim_x_train, y_train, batch_size=512, epochs=50, validation_split=0.12, verbose=1, callbacks=[es])



In [ ]:
model = Sequential()

model.add(Dense(400, input_dim=200))
model.add(Activation("relu"))
model.add(Dropout(0.1))
model.add(Dense(400))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dense(100))
model.add(Activation("relu"))
model.add(Dropout(0.1))
model.add(Dense(50))
model.add(Activation("relu"))

model.add(Dense(1, activation = 'sigmoid'))

optimizer = Nadam(1e-5)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

es = EarlyStopping(patience=10, verbose=1)
history = model.fit(X_train_scaled, y_train, batch_size=256, epochs=18, validation_split=0.2, verbose=2, callbacks=[es])



In [ ]:
testd = pd.read_csv('../input/test.csv')
pred_test = testd.iloc[:,1:]

ac = scaler.transform(pred_test)
ac_cnn = ac.reshape(pred_test.shape[0],pred_test.shape[1],1)
predictions = cmodel.predict_classes(ac_cnn)

ans = pd.DataFrame(predictions)
ans = ans[0]
test = pd.read_csv('../input/test.csv')
test_id = test["ID_code"]


In [ ]:
sub = pd.DataFrame()
sub['ID_code'] = test_id
sub['target'] = ans
sub.to_csv('submission_mcn5.csv', index=False)

In [ ]:
test_result = pd.read_csv('../working/submission_mcn5.csv')

